In [1]:

import tensorflow as tf
tf.enable_eager_execution()
from __future__ import absolute_import, division, print_function, unicode_literals


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import string

In [2]:
!ls

prunedCorpus.tar.gz  pruned_train.en  pruned_train.hi  sample_data


In [2]:
# download dataset
!wget 'http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/prunedCorpus.tar.gz'
!ls
# !unzip 'prunedCorpus.tar.gz'
!tar -xvzf 'prunedCorpus.tar.gz'

--2020-03-19 19:29:38--  http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/prunedCorpus.tar.gz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84265584 (80M) [application/x-gzip]
Saving to: ‘prunedCorpus.tar.gz’

prunedCorpus.tar.gz 100%[===================>]  80.36M   766KB/s    in 79s     

2020-03-19 19:30:58 (1.02 MB/s) - ‘prunedCorpus.tar.gz’ saved [84265584/84265584]

prunedCorpus.tar.gz  sample_data
pruned_train.en
pruned_train.hi


In [0]:

lines_hin = io.open('pruned_train.hi', encoding='UTF-8').read().strip().split('\n')
path_to_file = 'pruned_train.hi'

lines_eng = io.open('pruned_train.en', encoding='UTF-8').read().strip().split('\n')
path_to_file = 'pruned_train.en'

In [0]:
eng_sent = []
hin_sent = []
for i in range(len(lines_hin)):
  # splits = line.split('\t')
  #removing extra stuffs 
  eng = lines_eng[i]
  hin = lines_hin[i]
  # remove punctutation and make lower case
  eng = eng.translate(str.maketrans('', '', string.punctuation))
  eng = eng.lower()
  hin = hin.translate(str.maketrans('', '', string.punctuation))
  eng = '<start> ' + eng + ' <end>'
  hin = '<start> ' + hin + ' <end>'
  eng_sent.append(eng)
  hin_sent.append(hin)

In [0]:
eng_len = []
hin_len = []
y = []
for i in range(len(eng_sent)):
  eng_len.append(len(eng_sent[i]))
  hin_len.append(len(hin_sent[i]))
  y.append(i)


In [0]:
# plt.plot(eng_sent, y) 
# plt.show

# the above stunt didn't work as we can't plot 7 lakhs examples at once , now i'll look at alternatives

In [13]:
fifty = 0
hun = 0
twohun = 0
higher = 0

for i in eng_len:
  if i <= 50:
    fifty += 1
  elif i<=100:
    hun += 1
  elif i<=200:
    twohun += 1
  else:
    higher += 1
print(fifty,hun,twohun,higher)

fifty = 0
hun = 0
twohun = 0
higher = 0
for i in hin_len:
  if i <= 50:
    fifty += 1
  elif i<=100 and i>50:
    hun += 1
  elif i<=200 and i > 100:
    twohun += 1
  else: 
    higher += 1
print(fifty,hun,twohun,higher)

117016 296694 276972 97416
127300 300345 269980 90473


In [14]:
print(len(hin_len))

788098


# this is somewhat promising as we have more than 1 lakhs less than 50 words now let's check if both pairs are less than 50 or not

In [15]:
useable = 0
for i in range(len(eng_len)):
  if eng_len[i] < 50 and hin_len[i] < 50:
    useable += 1
print(useable)

90900


# Good good  90k examples with len less than 50 words let's use them and see if we can train the network then

In [0]:
eng_sent = []
hin_sent = []
for i in range(len(lines_hin)):
  # splits = line.split('\t')
  #removing extra stuffs 
  eng = lines_eng[i]
  hin = lines_hin[i]
  # remove punctutation and make lower case
  eng = eng.translate(str.maketrans('', '', string.punctuation))
  eng = eng.lower()
  hin = hin.translate(str.maketrans('', '', string.punctuation))
  eng = '<start> ' + eng + ' <end>'
  hin = '<start> ' + hin + ' <end>'
  if len(eng) <= 50:
    eng_sent.append(eng)
  if len(hin) <= 50:
    hin_sent.append(hin)